# Description
We use SVI to estimate IV and derivatives, as input to estimate Q

IV with R^2 > 0.99

In [1]:
import pandas as pd
import numpy as np
from src.quandl_data import get_btc_prices_2015_2022
import rpy2.robjects.packages as rpackages
from rpy2 import robjects
import matplotlib.pyplot as plt


def estimate_Q1(log_ret, IV, dIVdr, d2IVdr2, rf, tau, r, out_dir):
                            
    try:
        # MAKE SURE TO USE DIFFERENT 2ND INPUT FOR SP500!!
        moneyness, spd, logret, spd_logret, volas, cdf_m, cdf_ret, sigmas1, sigmas2= r.estimate_Q_from_IV(robjects.FloatVector(log_ret),
                                                                                        robjects.FloatVector(IV[0]), 
                                                                                        robjects.FloatVector(dIVdr), 
                                                                                        robjects.FloatVector(d2IVdr2), 
                                                                                        robjects.FloatVector(rf),
                                                                                        robjects.FloatVector([tau]),
                                                                                        robjects.StrVector([out_dir])
                                                                                       )
        # Can recover PD here as SPD/EPK
        spd_df = pd.DataFrame({'m': moneyness,
                               'spdy': spd,
                               'ret': logret,
                               'spd_ret': spd_logret,
                               'volatility': volas,
                               'cdf_m': cdf_m,
                               'cdf_ret': cdf_ret,
                               'sigma_prime': sigmas1,
                               'sigma_double_prime': sigmas2})

        return spd_df
    
    except Exception as e:
        print('exception: ', e)
        
        
# TTM
ttm = 27
tau = ttm / 365

# folder name 
out_dir = 'Results/Q_from_IV_0_1_3'

# load IV matrix
df = pd.read_csv('IV/interpolated IVs R2 0.99/interpolated_IV' + str(ttm) + '_99.csv')

# load interest rate
interest_rate_data = pd.read_csv('Data/IR_daily.csv')
interest_rate_data = interest_rate_data.rename(columns={'index': 'date','DTB3': 'interest_rate'})
interest_rate_data['interest_rate'] = interest_rate_data['interest_rate']/100

# log return
df_col_names = df.columns.to_numpy()
df_col_names = df_col_names[1:]
logret = df_col_names.astype(float)

# BTC price
btc_price = get_btc_prices_2015_2022()
btc_price[::-1].to_csv('Data/BTC_USD_Quandl_2015_2022.csv', index = False)
btc_price = btc_price.rename(columns={'Adj.Close': 'Close'})

# r package
base = rpackages.importr("base") 
r = robjects.r
r.source('src/Q_from_IV.R')


for i0 in range(df['Date'].size):
    date = df['Date'][i0]
    
    IV = df[df['Date']==date]
    IV = IV.drop(columns=['Date'])
    IV = IV.to_numpy()
    
    #dIVdr = df1[df1['Date']==date]
    #dIVdr = dIVdr.drop(columns=['Date'])
    #dIVdr = dIVdr.to_numpy()
    dIVdr = (IV[0][2:] - IV[0][:-2]) / (logret[2:] - logret[:-2])
    dIVdr = [float('nan')] + list(dIVdr) + [float('nan')]
    dIVdr[0] = (IV[0][1] - IV[0][0]) / (logret[1] - logret[0])
    dIVdr[-1] = (IV[0][-1] - IV[0][-2]) / (logret[-1] - logret[-2])


    #d2IVdr2 = df2[df2['Date']==date]
    #d2IVdr2 = d2IVdr2.drop(columns=['Date'])
    #d2IVdr2 = d2IVdr2.to_numpy()
    d2IVdr2 = (IV[0][2:] - 2 * IV[0][1:-1] + IV[0][:-2]) / ((logret[2:] - logret[:-2]) ** 2)
    d2IVdr2 = [float('nan')] + list(d2IVdr2) + [float('nan')]
    d2IVdr2[0] = (IV[0][2] - 2 * IV[0][1] + IV[0][0]) / ((logret[1] - logret[0]) ** 2)
    d2IVdr2[-1] = (IV[0][-1] - 2 * IV[0][-2] + IV[0][-3]) / ((logret[-1] - logret[-2]) ** 2)
    
    rf = interest_rate_data.interest_rate[interest_rate_data['date']==date].to_numpy()
    
    spd_btc = estimate_Q1(logret, IV, dIVdr, d2IVdr2, rf, tau, r, out_dir)
    
    try :
        spd_btc.to_csv(out_dir + '/btc_Q_'  + date +  '.csv')
        #print(spd_btc)
    
    except Exception as e:
        print(e)
        